In [16]:
#@title 1.3. IMPORT LIBRARY

import sys
sys.path.append('../utils')
from credit_base import *
from neural_net import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer, SimpleImputer

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy

from kan import *
torch.set_default_dtype(torch.float64)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cpu


In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
statlog_german_credit_data = fetch_ucirepo(id=144) 
  
# data (as pandas dataframes) 
X = statlog_german_credit_data.data.features 
y = statlog_german_credit_data.data.targets 
  
X.columns = ['Checking account', 'Duration', 'Credit history', 'Purpose',
            'Credit amount', 'Savings account', 'Employment since', 'Installment rate',
            'Marital status', 'Other debtors/guarantors', 'Residence since',
            'Property', 'Age', 'Other installments', 'Housing', 'Existing credits',
            'Job', 'Maintenance people', 'Telephone', 'Foreign worker']
# X.drop(columns=['Other installments', 'Maintenance people', 'Other debtors/guarantors'], inplace=True)
X = pd.get_dummies(X, dtype='int')

y['class'] = y['class'].map({1: 1, 2: 0})
y.columns = ['Risk']


X.head()

In [40]:
df = pd.concat([X, y], axis=1)
df.head()

,Duration,Credit amount,Installment rate,Residence since,Age,Existing credits,Maintenance people,Checking account_A11,Checking account_A12,Checking account_A13,...,Housing_A153,Job_A171,Job_A172,Job_A173,Job_A174,Telephone_A191,Telephone_A192,Foreign worker_A201,Foreign worker_A202,Risk
0,6,1169,4,4,67,2,1,1,0,0,...,0,0,0,1,0,0,1,1,0,1
1,48,5951,2,2,22,1,1,0,1,0,...,0,0,0,1,0,1,0,1,0,0
2,12,2096,2,3,49,1,2,0,0,0,...,0,0,1,0,0,1,0,1,0,1
3,42,7882,2,4,45,1,2,1,0,0,...,1,0,0,1,0,1,0,1,0,1
4,24,4870,3,4,53,2,2,1,0,0,...,1,0,0,1,0,1,0,1,0,0


In [41]:
def train_acc():
    return torch.mean((torch.argmax(model(dataset['train_input']), dim=1) == dataset['train_label']).type(dtype))

def test_acc():
    return torch.mean((torch.argmax(model(dataset['test_input']), dim=1) == dataset['test_label']).type(dtype))

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, logits, targets):
        probs = torch.softmax(logits, dim=1)  # Convert logits to probabilities
        probs_class_1 = probs[:, 1]  # Focus on class 1
        targets = targets.double()
        bce_loss = nn.BCELoss(reduction='none')(probs_class_1, targets)
        pt = torch.where(targets == 1, probs_class_1, 1 - probs_class_1)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        return focal_loss.mean()

criterion = FocalLoss(alpha=0.25, gamma=2)


In [49]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
n = df.shape[1] - 1

train_accuracies = []
test_accuracies = []

# Perform 5-fold stratified cross-validation
for fold, (train_idx, test_idx) in enumerate(skf.split(df, df['Risk'])):
    print(f"Fold {fold+1}/{5}")
    
    # Split the data for this fold
    df_train, df_test = df.iloc[train_idx], df.iloc[test_idx]
    X_train, y_train = df_train.drop(columns='Risk'), df_train['Risk']
    X_test, y_test = df_test.drop(columns='Risk'), df_test['Risk']
    
    # Prepare the dataset
    dataset = dict()
    dtype = torch.get_default_dtype()
    dataset['train_input'] = torch.from_numpy(X_train.values).type(dtype).to(device)
    dataset['train_label'] = torch.from_numpy(y_train.values).type(torch.long).to(device)
    dataset['test_input'] = torch.from_numpy(X_test.values).type(dtype).to(device)
    dataset['test_label'] = torch.from_numpy(y_test.values).type(torch.long).to(device)

    # Initialize model
    model = KAN(width=[n, 6, 2], grid=3, k=2, seed=42, device=device)

    # Train the model and evaluate it
    results = model.fit(dataset, opt="Adam", steps=1000, lr=0.001, lamb=0.001, metrics=(train_acc, test_acc), loss_fn=torch.nn.CrossEntropyLoss())

    print(f'Fold accuracy: {results['train_acc'][-1], results['test_acc'][-1]}')
    
    train_accuracies.append(results['train_acc'][-1])
    test_accuracies.append(results['test_acc'][-1])

# Output the average accuracies
print(f"Average Train Accuracy: {torch.mean(torch.tensor(train_accuracies)):.4f}")
print(f"Average Test Accuracy: {torch.mean(torch.tensor(test_accuracies)):.4f}")

Fold 1/5
checkpoint directory created: ./model
saving model version 0.0


description:   0%|                                                         | 0/1000 [00:00<?, ?it/s]

| train_loss: 6.69e-01 | test_loss: 6.97e-01 | reg: 2.86e+01 | : 100%|█| 1000/1000 [00:53<00:00, 18.


saving model version 0.1
Fold accuracy: (0.7875, 0.775)
Fold 2/5
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 6.64e-01 | test_loss: 7.19e-01 | reg: 2.91e+01 | : 100%|█| 1000/1000 [00:52<00:00, 19.


...

saving model version 0.1
Fold accuracy: (0.795, 0.74)
Fold 4/5
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 6.76e-01 | test_loss: 6.79e-01 | reg: 2.70e+01 | : 100%|█| 1000/1000 [00:51<00:00, 19.


saving model version 0.1
Fold accuracy: (0.78875, 0.795)
Fold 5/5
checkpoint directory created: ./model
saving model version 0.0


| train_loss: 6.71e-01 | test_loss: 7.06e-01 | reg: 2.62e+01 | : 100%|█| 1000/1000 [00:51<00:00, 19.

saving model version 0.1
Fold accuracy: (0.78875, 0.77)
Average Train Accuracy: 0.7900
Average Test Accuracy: 0.7680
